In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [3]:
batchsize=8

In [4]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'C:\Users\athar\Drowsiness_Detect\mrl-eye-dataset\data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'C:\Users\athar\Drowsiness_Detect\mrl-eye-dataset\data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 65340 images belonging to 2 classes.
Found 16335 images belonging to 2 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\athar\Drowsiness_Detect\mrl-eye-dataset\data\test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 3223 images belonging to 2 classes.


In [6]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 39, 39, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 39, 39, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 39, 39, 32)   0           ['batch_normalization[0][0]']

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [12]:
checkpoint = ModelCheckpoint(r'C:\Users\athar\Drowsiness_Detect\Model\model1.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [14]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=10)

Epoch 1/10


C:\Users\athar\AppData\Local\Temp\ipykernel_13624\3037644191.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


8167/8167 [==============================] - ETA: 0s - loss: 0.1944 - accuracy: 0.9243
Epoch 1: val_loss improved from inf to 0.21308, saving model to C:\Users\athar\Drowsiness_Detect\Model\model1.h5
8167/8167 [==============================] - 713s 87ms/step - loss: 0.1944 - accuracy: 0.9243 - val_loss: 0.2131 - val_accuracy: 0.9177 - lr: 0.0010
Epoch 2/10
8166/8167 [============================>.] - ETA: 0s - loss: 0.1736 - accuracy: 0.9328
Epoch 2: val_loss improved from 0.21308 to 0.19515, saving model to C:\Users\athar\Drowsiness_Detect\Model\model1.h5
8167/8167 [==============================] - 515s 63ms/step - loss: 0.1736 - accuracy: 0.9328 - val_loss: 0.1951 - val_accuracy: 0.9231 - lr: 0.0010
Epoch 3/10
8167/8167 [==============================] - ETA: 0s - loss: 0.1659 - accuracy: 0.9363
Epoch 3: val_loss did not improve from 0.19515
8167/8167 [==============================] - 542s 66ms/step - loss: 0.1659 - accuracy: 0.9363 - val_loss: 0.2015 - val_accuracy: 0.9209 - lr: 

In [ ]:
# Model Evaluation

In [44]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

Instructions for updating:
Please use Model.evaluate, which supports generators.


KeyboardInterrupt: 

In [ ]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

In [ ]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)